In [122]:
import pandas as pd
import numpy as np
import librosa
import h5py
import math
import warnings
import json
import importlib
import loop_generator

importlib.reload(loop_generator)

from loop_generator import LoopGenerator

In [37]:
one_to_loop_ratio = 1
loop_unit_multi_sample_pct = 0.2

In [38]:
meta_data = pd.read_csv('audio_metadata - filtered_copy.csv', sep=',', index_col=False)
f = open('patterns.json')
looping_patterns = json.load(f)
# meta_data.drop(columns=['group', 'category', 'sub_category'], inplace=True)

In [39]:
df_id = np.array(meta_data['id'].astype(int)).flatten()

wavs = []
srs = []

def load_sample(f, sample_id):
    group = f["audio_data"][str(sample_id)]
    waveform = np.array(group["waveform"])
    sample_rate = np.array(group["sample_rate"])
    return waveform, sample_rate


with h5py.File('filtered_audio_data.h5', "r") as f:
    for i in df_id:
        waveform, sr = load_sample(f, i)  # load sample with id=42
        wavs.append(waveform)
        srs.append(sr)


samples = pd.DataFrame({'id': df_id, 'waveform': wavs, 'sample_rate': srs, })

In [47]:
#samples = samples.set_index('id').join(meta_data.set_index('id'))
#sampis = samples.copy()
samples = samples.join(meta_data.set_index('id'), on='id')
#one_shots = samples.copy()
#loops = samples.copy()

In [48]:
category_counts = {}
sample_categories = set(samples['category'])
for cat in sample_categories:
    category_counts[cat] = {}
    category_counts[cat]["loop_count"] = len(samples[(samples['category'] == cat) & (samples['One_Shot'] == 0)])
    category_counts[cat]["one_shot_count"] = len(samples[(samples['category'] == cat) & (samples['One_Shot'] == 1)])
    if 'One_Shot_Intent' and 'Loop_id' in samples.columns:
        category_counts[cat]['one_shot_intent_count'] = len(samples[(samples['category'] == cat) & (samples['One_Shot'] == 1) & (samples['One_Shot_Intent'] == 1)])
        category_counts[cat]['loop_intent_count'] = len(samples[(samples['category'] == cat) & (samples['One_Shot'] == 0) & (samples['Loop_id'].notnull())])
    else:
        category_counts[cat]['one_shot_intent_count'] = 0
        category_counts[cat]['loop_intent_count'] = 0
    category_counts[cat]["all"] = category_counts[cat]["loop_count"] + category_counts[cat]["one_shot_count"]
    category_counts[cat]["required_one_shot_count_samples"] = math.floor((one_to_loop_ratio * category_counts[cat]["all"]) / (one_to_loop_ratio +1 + loop_unit_multi_sample_pct))
    category_counts[cat]["required_loop_count_units"] = math.floor((one_to_loop_ratio * category_counts[cat]["all"]) / (one_to_loop_ratio +1 + loop_unit_multi_sample_pct))
    category_counts[cat]["required_loop_count_samples"] = int(category_counts[cat]["required_loop_count_units"] * (1+loop_unit_multi_sample_pct))
    if category_counts[cat]["required_loop_count_samples"] + category_counts[cat]["required_one_shot_count_samples"] > category_counts[cat]["all"]:
        warnings.warn(f"greater required samples than available")


category_counts

{'Clap': {'loop_count': 195,
  'one_shot_count': 1005,
  'one_shot_intent_count': 0,
  'loop_intent_count': 0,
  'all': 1200,
  'required_one_shot_count_samples': 545,
  'required_loop_count_units': 545,
  'required_loop_count_samples': 654},
 'Hi Hat': {'loop_count': 195,
  'one_shot_count': 1005,
  'one_shot_intent_count': 0,
  'loop_intent_count': 0,
  'all': 1200,
  'required_one_shot_count_samples': 545,
  'required_loop_count_units': 545,
  'required_loop_count_samples': 654},
 'Kick': {'loop_count': 195,
  'one_shot_count': 1005,
  'one_shot_intent_count': 0,
  'loop_intent_count': 0,
  'all': 1200,
  'required_one_shot_count_samples': 545,
  'required_loop_count_units': 545,
  'required_loop_count_samples': 654}}

In [49]:
samples_copy = samples.copy()
if 'One_Shot_Intent' and 'Loop_id' not in samples_copy.columns:
    samples_copy['One_Shot_Intent'] = np.nan
    samples_copy['Loop_id'] = np.nan

# next up  
made a way to import files. get their category and count by one shot or loop  

now i need to do the following:  
- figure out the number of samples to switch from one shot to loop in order to have a 50/50 split: done  
  - a small amount (ex: 20%) of loops should use more than 1 sample. need to take into account that we'll use a higher amount of samples for loops than oneshots to acheive a 50/50 split: done  
- identify one shot intent for each oneshot: work in progress  
- doccument their intent
- all of this needs to work with the possibility that loops may or may not exist. can't overwrite existing data when making loops. loop ids can't clash with existing loop ids



# line to save for when 
keep this line for when i need to start generating new ids for the loops
loop_id_num = max(loops['loop_id'])

In [123]:
gen = LoopGenerator()
print(gen.df)

Empty DataFrame
Columns: []
Index: []


In [124]:
num_of_loop_units = int(one_to_loop_ratio / loop_unit_multi_sample_pct)
num_of_loop_samples = math.ceil((one_to_loop_ratio + loop_unit_multi_sample_pct) / loop_unit_multi_sample_pct)

# if one shot count is greater than needed and 
for cat in category_counts:
    # count for intents meet the required counts - no edits need to be made
    if (category_counts[cat]['one_shot_intent_count'] == category_counts[cat]['required_one_shot_count_samples']) and (category_counts[cat]['loop_intent_count'] == category_counts[cat]['required_loop_count_samples']):
        print(f'one shot intent and loop intent counts for {cat} meet requirements')
    # 0 count for both intents - just created intent col
    elif category_counts[cat]['one_shot_count'] > category_counts[cat]['required_one_shot_count_samples'] and category_counts[cat]['one_shot_intent_count'] == 0 and category_counts[cat]['loop_intent_count'] == 0 :
        tmp = samples_copy[(samples_copy['category'] == cat) & (samples_copy['One_Shot'] == 1)].copy()
        ones = tmp.iloc[:category_counts[cat]['required_one_shot_count_samples']].copy()
        ones['One_Shot_Intent'] = 1
        
        
        existing_loops = samples_copy[(samples_copy['category'] == cat) & (samples_copy['One_Shot'] == 0)].copy()
        loops = tmp.iloc[category_counts[cat]['required_one_shot_count_samples']:(category_counts[cat]['required_one_shot_count_samples']+category_counts[cat]['required_loop_count_samples']-len(existing_loops))].copy()
        

        existing_loops['One_Shot_Intent'] = 0
        existing_loops['Loop_id'] = existing_loops['id']
        loops['One_Shot_Intent'] = 0
        loops.reset_index(drop=False, inplace=True)
        #print(loops)
        
        nums = []
        for index, row in loops.iterrows():
            nums.append(index)
            if index % num_of_loop_samples != 0:
                #print(nums)
                #print(nums[0], nums[-1])
                #print(loops[nums[0]:nums[-1]+1]['waveform'])
                #print(loops[nums[0]:nums[-1]+1]['sample_rate'].to_list())
                loop = gen.generate_loop(looping_patterns[cat][index % len(looping_patterns[cat])], loops[nums[0]:nums[-1]+1]['waveform'].to_list(), loops[nums[0]:nums[-1]+1]['sample_rate'].to_list(), np.random.randint(70, 180), cat)
                nums = []
            

                



        # need to join every file back to samples_copy using id
        #print(f'category: {cat} - - tmp len: {len(tmp)} - - ones len: {len(ones)} - - loops len: {len(loops)}')
    # non 0 count exists for intents. intents already existing - dataset has done some loop generation already and new files were probably added
    elif (category_counts[cat]['one_shot_count'] > category_counts[cat]['required_one_shot_count_samples']) and (category_counts[cat]['one_shot_intent_count'] < category_counts[cat]['required_one_shot_count_samples']) and (category_counts[cat]['loop_intent_count'] < category_counts[cat]['required_loop_count_samples']):
        tmp = samples_copy[samples_copy['category'] == cat].copy()
        used_ones = tmp[(tmp['One_Shot'] == 1) & (tmp['One_Shot_Intent'] == 1)]
        unused_ones = tmp[(tmp['One_Shot'] == 1) & (tmp['One_Shot_Intent'].isnull())]
        print(f'used_ones len: {len(used_ones)} -- unused_ones len: {len(unused_ones)}')
    elif category_counts[cat]['needed_each'] > category_counts[cat]['one_shot_count']:
        warnings.warn(f"not enough one shots to reach {one_to_loop_ratio}:1 ratio")



[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -1.52587891e-05
 -1.52587891e-05 -1.52587891e-05]
loops
/loops

loops/Clap
[0.01644897 0.14105225 0.4381714  ... 0.         0.         0.        ]
loops
/loops

loops/Clap
[-0.06407166 -0.01208496  0.13417053 ...  0.          0.
  0.        ]
loops
/loops

loops/Clap
[-0.01379395 -0.03186035 -0.07449341 ...  0.          0.
  0.        ]
loops
/loops

loops/Clap
[0. 0. 0. ... 0. 0. 0.]
loops
/loops

loops/Clap
[0. 0. 0. ... 0. 0. 0.]
loops
/loops

loops/Clap
[0. 0. 0. ... 0. 0. 0.]
loops
/loops

loops/Clap
[0.         0.         0.         ... 0.00025928 0.00031435 0.00023538]
loops
/loops

loops/Clap
[-3.9100647e-05 -3.9100647e-05 -3.9100647e-05 ...  0.0000000e+00
  0.0000000e+00  0.0000000e+00]
loops
/loops

loops/Clap
[-3.2186508e-06 -3.2186508e-06 -3.2186508e-06 ...  0.0000000e+00
  0.0000000e+00  0.0000000e+00]
loops
/loops

loops/Clap
[0. 0. 0. ... 0. 0. 0.]
loops
/loops

loops/Clap
[0. 0. 0. ... 0. 0. 0.]
loops
/loops

loops/Cl

In [125]:
loops.loc[2:2]

,index,id,waveform,sample_rate,file_path,file_name,group,category,sub_category,Reverse,...,Vibraslap,Timpani,Vocal FX,China,Cowbell,Bell,Orchestra,Metalic Tap,One_Shot_Intent,Loop_id
2,2368,22620,"[-8.2969666e-05, -0.004480183, -0.03542751, -0...",44100,sounds of kshmr vol 3/kshmr_drums/kshmr_kicks/...,KSHMR_Hardstyle_Kick_13_D#.wav,Drum,Kick,NaN,0,...,0,0,0,0,0,0,0,0,0,NaN


In [65]:
ones.head()

,waveform,sample_rate,file_path,file_name,group,category,sub_category,Reverse,Forward,One_Shot,...,Vibraslap,Timpani,Vocal FX,China,Cowbell,Bell,Orchestra,Metalic Tap,One_Shot_Intent,Loop_id
id,,,,,,,,,,,,,,,,,,,,,
5487,"[5.164626e-05, -0.000103295286, 2.5824858e-05,...",44100,808 and kick/heavy kick.wav,Heavy Kick.wav,Drum,Kick,NaN,0,1,1,...,0,0,0,0,0,0,0,0,1,NaN
5087,"[-0.0004272461, -0.0016479492, -0.007232666, -...",44100,808 samples/808/bass (12).wav,bass (12).wav,Drum,Kick,NaN,0,1,1,...,0,0,0,0,0,0,0,0,1,NaN
5084,"[-0.0009765625, -0.0026550293, -0.009033203, -...",44100,808 samples/808/bass (13).wav,bass (13).wav,Drum,Kick,NaN,0,1,1,...,0,0,0,0,0,0,0,0,1,NaN
5083,"[-0.00039672852, -0.00045776367, -0.0014953613...",44100,808 samples/808/bass (14).wav,bass (14).wav,Drum,Kick,NaN,0,1,1,...,0,0,0,0,0,0,0,0,1,NaN
5098,"[-0.0029296875, -0.010894775, -0.051208496, -0...",44100,808 samples/808/bass (16).wav,bass (16).wav,Drum,Kick,NaN,0,1,1,...,0,0,0,0,0,0,0,0,1,NaN
